In [1]:
# constructing accumulibacter_model

from cobra import Model, Reaction, Metabolite
import cobra.io
import pandas as pd

#Create an empty model
model = Model("Accumulibacter_anaerobic")

# Define metabolites
met_ids = [
    'Mal4', 'Pi', 'G1P', 'Mal3', 'G6P', 'F6P', 'ATP', 'FBP', 'ADP',
    'DHAP', 'G3P', 'NAD', 'BPG', 'NADH', 'P3G', 'P2G', 'PEP', 'H2O', 'PYR',
    'CoA', 'AcCoA', 'PPi', 'HB', 'HBCoA', 'AACoA', 'X5P', 'E4P', 'SBP',
    'S7P', 'Ri5P', 'Ru5P', 'RBP', 'CO2', 'PolyPP', 'PolyP', 'AMP', 'PHB',
    'Ace'
]

mets = {m_id: Metabolite(m_id, name=m_id, compartment="c") for m_id in met_ids}
model.add_metabolites(mets.values())

# Define and add reactions
rxns = [
    ("GlycP",  "Mal4 + Pi <=> G1P + Mal3"),
    ("GPM",    "G1P <=> G6P"),
    ("PGI",    "G6P <=> F6P"),
    ("ATPPFK","F6P + ATP <=> FBP + ADP"),
    ("FBPald","FBP <=> DHAP + G3P"),
    ("TPI",    "DHAP <=> G3P"),
    ("GAPDH",  "G3P + NAD + Pi <=> BPG + NADH"),
    ("PGK",    "BPG + ADP <=> P3G + ATP"),
    ("PGM",    "P3G <=> P2G"),
    ("ENO",    "P2G <=> PEP + H2O"),
    ("PYK",    "PEP + AMP + PPi <=> ATP + PYR + Pi"),
    ("PDH",    "PYR + NAD + CoA <=> AcCoA + NADH + CO2"),
    ("ACS",    "Ace + CoA + ATP <=> AcCoA + PPi + AMP"),
    ("Thio",   "AcCoA + AcCoA <=> AACoA + CoA"),
    ("AAR",    "AACoA + NADH <=> HBCoA + NAD"),
    ("PHBsyn", "HBCoA + HB <=> PHB + CoA"),
    ("TktA",   "F6P + G3P <=> X5P + E4P"),
    ("SBPald", "DHAP + E4P <=> SBP"),
    ("SBPase", "SBP + H2O <=> S7P + Pi"),
    ("TktB",   "G3P + S7P <=> Ri5P + X5P"),
    ("RibE",   "X5P <=> Ru5P"),
    ("RibI",   "Ri5P <=> Ru5P"),
    ("RbuK",   "Ru5P + ATP <=> RBP + ADP"),
    ("RbuCO",  "RBP + CO2 + H2O <=> P3G + P3G"),
    ("ADPPPT", "PolyPP + ADP <=> ATP + PolyP"),
    ("AMPPPT", "PolyPP + AMP <=> ADP + PolyP"),
    ("PPiase", "PPi + H2O <=> Pi + Pi")
]

for rxn_id, formula in rxns:
    rxn = Reaction(rxn_id)
    model.add_reactions([rxn])
    rxn.reaction = formula

# Add exchange reactions with specified bounds
ex_bounds = {
    "PolyPP": (-50, 50),
    "PolyP":  (-50, 50),
    "Mal4":   (-50, 50),
    "Mal3":   (-50, 50),
    "Ace":    (-50, 50),
    "Pi":     (0, 50),
    "PPi":    (0, 50),
    "CO2":    (-50, 50),
    "HB":     (-50, 50),
    "PHB":    (-50, 50),
    "H2O":    (-50, 50),
}

for met_id, (lb, ub) in ex_bounds.items():
    ex = Reaction(f"EX_{met_id}")
    ex.lower_bound = lb
    ex.upper_bound = ub
    ex.add_metabolites({mets[met_id]: -1})
    model.add_reactions([ex])

# Set the objective function
model.objective = "EX_PHB"

# Save model in different formats
#cobra.io.save_matlab_model(model, "Accumulibacter_anaerobic.mat")
cobra.io.save_json_model(model, "Accumulibacter_anaerobic.json")

# Save reaction formulas to CSV
df = pd.DataFrame({
    "Reaction ID": [rxn.id for rxn in model.reactions],
    "Formula": [rxn.reaction for rxn in model.reactions]
})
df.to_csv("Accumulibacter_anaerobic_stoichiometric_model.csv", index=False)